In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
from IPython.display import display, Image
import numpy as np
import matplotlib.pyplot as plt

letter=['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y']

def take_photo(quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  
  data = eval_js('takePhoto({})'.format(quality))
  img = js_to_image(data)

  # Preprocess the frame
  frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
  frame = cv2.resize(frame, (28, 28))  # Resize to match the input size of the model
  frame = np.expand_dims(frame, axis=-1)  # Add an extra dimension to match the model's input shape
  frame = frame / 255.0  # Normalize pixel values between 0 and 1
  print(frame.shape)
  plt.imshow(frame.reshape(28,28),cmap='gray')
  plt.axis('off')
  plt.show()

  from tensorflow.keras.models import load_model
  model=load_model("/content/model_1.h5")


    # Perform inference
  prediction = model.predict(np.array([frame]))
  predicted_class = np.argmax(prediction)

    # Get the predicted sign label
  sign_label = letter[predicted_class]
  print(sign_label)

    # Display the frame and predicted sign label
  # cv2.putText(frame, sign_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
  
  # cv2.imshow('Sign Language Recognition', frame)

  # cv2.imwrite(filename, img)

  # return filename
take_photo()